In [ ]:
import sys
sys.path.insert(0,'py_files')

def collect_train_test(pad_size, root_dir = "../Data/training/"):
    ''' Load images and pad them using mirror boundary conditions
    '''
    # Load a set of image
    image_dir = root_dir + "images/"
    files = os.listdir(image_dir)
    n = len(files) 
    print("Loading " + str(n) + " images")
    imgs = [load_image(image_dir + files[i]) for i in range(n)]

    gt_dir = root_dir + "groundtruth/"
    print("Loading " + str(n) + " groundtruth images")
    gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

    print('Padding images using pad of: ', pad_size)
    imgs = padding_imgs(np.array(imgs),pad_size)
    gt_imgs = padding_GT(np.array(gt_imgs),pad_size)
    
    print('Shape of imgs: ',imgs.shape)
    print('Shape of gt_imgs: ',imgs.shape)
    return imgs, gt_imgs

def predict_on_test_and_save_prediction(root = '../Data'):
    '''This function does the prediction, generate a submission file and saves the predicted images.
        NOTE: works for a model with 2 neurons at the last layer'''
    
    print('Loading test images')
    test_images = np.asarray(pick_test_images(root))
    test_images = padding_imgs(np.array(test_images),pad_size)
    
    #prepare the input for the prediction
    test_inputs = imgs_to_windows(test_images,608,patch_size,input_size)
    print('Inputs for the test are ready. Shape: ', test_inputs.shape)
    
    print('Predicting... ')
    #recover the model
    model, _,_ = create_model()
    model.load_weights(NameWeights)
    test_prediction = model.predict(test_inputs)
    print('Done!')
    
    print('Generating submission file...')
    test_predicted_labels = ((test_prediction[:,0] < test_prediction[:,1]) * 1 ).flatten()
    test_labels = test_predicted_labels.reshape(50,-1)
    MY_masks_to_submission(SubmissionName, test_labels)
    print('Submission saved in: ', SubmissionName)
    
    import pickle
    # Saving the objects:
    with open(PredictionName, 'wb') as f:
        pickle.dump(test_predicted_labels, f)
    print('Prediction saved in ', PredictionName)
    
    
def train_f1_score():
    #choose some images
    IDX_VALID = np.random.randint(low=0, high=100, size = 4)
    N_valid = IDX_VALID.shape[0]


    #####################################################################
    root_dir = '../Data/training/'
    #Load, pad onnly imgs using the real pad_size
    image_dir = root_dir + "images/"
    files = os.listdir(image_dir)
    n = len(files) 
    imgs = [load_image(image_dir + files[i]) for i in range(n)]
    gt_dir = root_dir + "groundtruth/"
    gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
    imgs = padding_imgs(np.array(imgs),pad_size)
    gt_imgs = np.asarray(gt_imgs)


    #Create X and Y validation
    X_valid = imgs[IDX_VALID]
    val_inputs = imgs_to_windows(X_valid,400,patch_size,input_size)

    Y_valid = gt_imgs[IDX_VALID]
    val_gt_patches = [img_crop(Y_valid[i], patch_size, patch_size) for i in range(N_valid)]
    val_gt_patches =  np.asarray([val_gt_patches[i][j] for i in range(len(val_gt_patches)) for j in range(len(val_gt_patches[i]))])
    val_true_labels = np.asarray([value_to_class(np.mean(val_gt_patches[i])) for i in range(len(val_gt_patches))])


    #Predict
    print('Predicting... ')
    val_prediction = model.predict(val_inputs)
    print('Done!')

    #Compute accuracy and f1_Score
    from sklearn.metrics import f1_score
    val_predicted_labels = ((val_prediction[:,0] < val_prediction[:,1]) * 1 ).flatten()
    val_accuracy = np.sum(np.abs(val_predicted_labels - val_true_labels))/val_true_labels.shape[0]
    print('Accuracy on validation set is: ', 1 - val_accuracy)
    print('F1 score on validation set is:', f1_score(val_true_labels,val_predicted_labels))


In [ ]:
from model_5 import *

In [ ]:
x_train, y_train = collect_train_test(pad_rotate_size)

In [ ]:
model = train(x_train,y_train)

In [ ]:
predict_on_test_and_save_prediction()

In [ ]:
train_f1_score()

In [ ]:
from model_5_bis import *

In [ ]:
x_train, y_train = collect_train_test(pad_rotate_size)

In [ ]:
model = train(x_train,y_train)

In [ ]:
predict_on_test_and_save_prediction()

In [ ]:
train_f1_score()